In [1]:
!pip install nltk sklearn pandas numpy matplotlib seaborn flask
!pip install tensorflow keras


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import pandas as pd


df = pd.read_csv("Liar_Dataset.csv")
print(df.head())


    [ID].json       label                                          statement  \
0  11972.json        TRUE  Building a wall on the U.S.-Mexico border will...   
1  11685.json       FALSE  Wisconsin is on pace to double the number of l...   
2  11096.json       FALSE  Says John McCain has done nothing to help the ...   
3   5209.json   half-true  Suzanne Bonamici supports a plan that will cut...   
4   9524.json  pants-fire  When asked by a reporter whether hes at the ce...   

                                          subject(s)  \
0                                        immigration   
1                                               jobs   
2                    military,veterans,voting-record   
3  medicare,message-machine-2012,campaign-adverti...   
4  campaign-finance,legal-issues,campaign-adverti...   

                            speaker   speaker's job title state info  \
0                        rick-perry              Governor      Texas   
1                 katrina-shankland  S

In [4]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.split()  # Tokenize
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    return " ".join(text)

df["clean_text"] = df["statement"].apply(clean_text)
print(df["clean_text"].head())


0    building wall u mexico border take literally y...
1            wisconsin pace double number layoffs year
2              says john mccain done nothing help vets
3    suzanne bonamici supports plan cut choice medi...
4    asked reporter whether hes center criminal sch...
Name: clean_text, dtype: object


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["clean_text"]).toarray()
y = df["label"].apply(lambda x: 1 if x == "TRUE" else 0)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_train, y_train)


y_pred = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))


Random Forest Accuracy: 0.8295543393275997


In [7]:
from sklearn.svm import SVC

svm_model = SVC(kernel="linear")
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))


SVM Accuracy: 0.8373729476153244


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df["clean_text"])

X_seq = tokenizer.texts_to_sequences(df["clean_text"])
X_padded = pad_sequences(X_seq, maxlen=100)  # Padding to same length


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_padded, y, test_size=0.2, random_state=42)
model.fit(X_train_lstm, y_train_lstm, epochs=5, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8426 - loss: 0.4633 - val_accuracy: 0.8374 - val_loss: 0.4432
Epoch 2/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8403 - loss: 0.4245 - val_accuracy: 0.8374 - val_loss: 0.4497
Epoch 3/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8530 - loss: 0.3322 - val_accuracy: 0.8182 - val_loss: 0.5052
Epoch 4/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9008 - loss: 0.2438 - val_accuracy: 0.8084 - val_loss: 0.5498
Epoch 5/5
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9292 - loss: 0.1827 - val_accuracy: 0.8002 - val_loss: 0.6492


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(128, return_sequences=True),
    LSTM(64),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_padded, y, test_size=0.2, random_state=42)
model.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))


Epoch 1/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.8289 - loss: 0.4553 - val_accuracy: 0.8374 - val_loss: 0.4429
Epoch 2/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8422 - loss: 0.4116 - val_accuracy: 0.8382 - val_loss: 0.4507
Epoch 3/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8625 - loss: 0.3227 - val_accuracy: 0.8268 - val_loss: 0.5036
Epoch 4/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9100 - loss: 0.2289 - val_accuracy: 0.7889 - val_loss: 0.5832
Epoch 5/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9362 - loss: 0.1623 - val_accuracy: 0.7842 - val_loss: 0.6930
Epoch 6/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9638 - loss: 0.0989 - val_accuracy: 0.7631 - val_loss: 0.9206
Epoch 7/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9798 - loss: 0.0639 - val_accuracy: 0.7674 - val_loss: 1.1131
Epoch 8/20
320/320 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9858 - loss: 0.0422 - val_accu

In [11]:
import pickle

# Save the model
pickle.dump(rf_model, open("fake_news_model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))


In [12]:
import pickle


with open("random_forest_model.pkl", "wb") as model_file:
    pickle.dump(rf_model, model_file)


with open("tfidf_vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print("Random Forest Model and Vectorizer Saved Successfully!")


Random Forest Model and Vectorizer Saved Successfully!


In [13]:

model.save("lstm_fake_news_model.h5")
print("LSTM Model Saved Successfully!")


LSTM Model Saved Successfully!


In [14]:
from google.colab import files


files.download("random_forest_model.pkl")


files.download("tfidf_vectorizer.pkl")


files.download("lstm_fake_news_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>